In [2]:
import numpy as np
from scipy import linalg 
import os
import sys

def turnbits_rec(p):
    if(p==1):
        return np.array([[True, False],[True, True]]) #第一列的True是截距项
    else:
        tmp1 = np.c_[ turnbits_rec(p-1), np.array([False]*(2**(p-1))).reshape((2**(p-1),1))]
        tmp2 = np.c_[ turnbits_rec(p-1), np.array([True]*(2**(p-1))).reshape((2**(p-1),1))]
        return np.r_[tmp1, tmp2]
def solve_sym(xtx, xty):
        L = linalg.cholesky(xtx)
        return linalg.lapack.dpotrs(L, xty)[0]

In [3]:
x = np.loadtxt("C:/Users/Effle Hu/AnancondaProjects/0306/Best Subset Regression/prostate/x.txt", delimiter=",")
y = np.loadtxt("C:/Users/Effle Hu/AnancondaProjects/0306/Best Subset Regression/prostate/y.txt", delimiter=",")
names = np.loadtxt("C:/Users/Effle Hu/AnancondaProjects/0306/Best Subset Regression/prostate/names.txt", delimiter=",", dtype=str)

In [45]:
class BestSubsetReg(object):
    
    def __init__(self, x=0, y=0, names=0, inter=True, isCp=True, isAIC=True):
        self.n, self.p = np.shape(x)
        self.names = names[0:self.p]
        self.isCp = isCp
        self.isAIC = isAIC
        if inter:
            self.x = np.c_[np.ones((self.n,1)), x]
            self.p1 = self.p+1
            self.names = np.append(["inter"],self.names)
        else:
            self.x = x
            self.p1 = self.p
        self.y = y
        self.xx = np.dot(self.x.T, self.x)
        self.xy = np.dot(self.x.T, self.y)
        self.ind_var = turnbits_rec(self.p)
        self.b = [solve_sym(self.xx[ind][:,ind], self.xy[ind]) for ind in self.ind_var]
        if self.isCp:
            self.Cp = 0
        if self.isAIC:
            self.AIC = 0
        self.cv_err = 0
     
    def Cp_AIC(self):
        XTy_1 = [self.xy[ind] for ind in self.ind_var]    
        yTy = np.dot(self.y.T, self.y)
        def RSS(xty, beta):
            return np.sum(yTy - np.dot(xty.T,beta))
        rss = np.array(list(map(RSS, XTy_1, self.b)))
        sigma2 = rss[-1]/(self.n - self.p1)
        d = [len(self.b[i]) for i in np.arange(0,2**self.p)]
        if self.isCp:
            self.Cp = [rss[i]+2*sigma2*d[i] for i in np.arange(0,2**self.p)]
            min_id = np.argmin(self.Cp)
            print("以Cp为准则：")
            print(list(zip(self.names[self.ind_var[min_id][0:]],self.b[min_id])))
            
        if self.isAIC:
            self.AIC = [self.n*np.log(rss[i]) + 2*d[i] for i in np.arange(0,2**self.p)]
            min_id = np.argmin(self.AIC)
            print("以AIC为准则：")
            print(list(zip(self.names[self.ind_var[min_id][0:]],self.b[min_id])))
    
    def CV(self,K=10):
        indexs = np.array_split(np.random.permutation(np.arange(0,self.n)),K)
        def cvk(ind,index):
            xtx_k = self.xx[ind][:,ind] - np.dot((self.x[index][:,ind]).T, self.x[index][:,ind])
            xty_k = self.xy[ind] - np.dot((self.x[index][:,ind]).T, self.y[index])
            b_k = solve_sym(xtx_k, xty_k)
            return np.sum((self.y[index] - np.dot(self.x[index][:,ind], b_k))**2)
         #self.cv_err = np.sum(np.array([[cvk(ind, index) for ind in self.ind_var] for index in indexs]), axis=0)/self.n
        self.cv_err = [(np.sum(np.array([cvk(ind, index) for index in indexs])))/self.n for ind in self.ind_var]
        min_id = np.argmin(self.cv_err)
        print("交叉验证法：")
        print(list(zip(self.names[self.ind_var[min_id][0:]],self.b[min_id])))
        

In [50]:
reg1 = BestSubsetReg(x, y,names)
reg1.Cp_AIC()
reg1.CV(K=30)

以Cp为准则：
[('inter', 0.49472926218273405), ('lcavol', 0.5439978569443544), ('lweight', 0.58821270309517), ('age', -0.01644484649754566), ('lbph', 0.10122333372346587), ('svi', 0.714903976347165)]
以AIC为准则：
[('inter', 0.49472926218273405), ('lcavol', 0.5439978569443544), ('lweight', 0.58821270309517), ('age', -0.01644484649754566), ('lbph', 0.10122333372346587), ('svi', 0.714903976347165)]
交叉验证法：
[('inter', -0.77715664158), ('lcavol', 0.5258518819809425), ('lweight', 0.6617699115944506), ('svi', 0.6656665628572002)]
